In [ ]:
import numpy as np 
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import os
from tensorflow.keras import models, layers, activations, optimizers, utils, losses, initializers, metrics, callbacks

In [ ]:
path = '/kaggle/input/melanoma-cancer-dataset/train'
epochs = 100
batch_size = 32
patience = 5
learning_rate = 0.001 #alpha
model_path = '/kaggle/input/checkpoints/model.keras'

In [ ]:
model = models.Sequential([
    layers.Resizing(56, 56),
    layers.Rescaling(1.0/255),
    layers.RandomRotation((-0.2, 0.2)),

    layers.Conv2D(32, (3, 3), # 32 @ 54x54
         activation = 'relu',
         kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2, 2)), # 32 @ 27x27

    layers.Conv2D(64, (3, 3), # 2048 @ 24x24
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2, 2)), # 2048 @ 8x8

    layers.Flatten(), # ~125k parametros

    layers.Dropout(0.2), # 20% dos neurônios serão desativados
    layers.Dense(128,
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),

    layers.Dense(64,
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),

    layers.Dense(64,
        activation = 'relu', #activations.relu()
        kernel_initializer = initializers.RandomNormal()
    ),

    layers.Dense(1,
        activation = 'sigmoid',
        kernel_initializer = initializers.RandomNormal()
    ),
                 
])

In [ ]:
model.compile(
    optimizer = optimizers.Adam(
        learning_rate = learning_rate
    ),
    loss = losses.BinaryCrossentropy(),
    metrics = [ 
        metrics.BinaryAccuracy(), 
        metrics.Precision(), 
        metrics.Recall()
    ]
)

In [ ]:
train = utils.image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset = "training",
    shuffle = True,
    seed = 423,
    image_size = (224, 224),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset = "validation",
    shuffle = True,
    seed = 42,
    image_size = (224, 224),
    batch_size = batch_size
)

In [ ]:
model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        # callbacks.ModelCheckpoint(
        #     filepath = model_path,
        #     save_weights_only = False,
        #     monitor = 'loss',
        #     mode = 'min',
        #     save_best_only = True
        # )
    ]
)